# Simple loop agent

In this example, we will create an loop agent that simply generate a description base on a same query and get all results

In other word, this is a "for loop" agent.

In [1]:
from collections.abc import Sequence
from typing import List

from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import AgentMessage, BaseAgent
from aap_core.chain import BaseLLMChain
from aap_core.orchestration import LoopAgent
from aap_core.types import AgentResponse
from aap_llamaindex.chain import ChatCausalMultiTurnsChain
from llama_index.llms.ollama import Ollama

In [2]:
class Agent(BaseAgent):
    chain: BaseLLMChain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.card.name
        self.state = "idle"
        return message

In [3]:
from collections.abc import Generator

system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

def iter(max_iteration: int) -> Generator[bool, None, None]:
    n = 0
    while True:
        yield n >= max_iteration
        n = (n + 1) % (max_iteration + 1)

model = Ollama(model="llama3.2:latest", base_url="192.168.55.1:11434", request_timeout=400, context_window=32768, temperature=1.5)
chain = ChatCausalMultiTurnsChain(model=model, system_prompt=system_prompt, user_prompt_template=user_prompt)
writer_skill = AgentSkill(
    id='writer-skill',
    name="writer skill",
    description="writer skill",
    tags=['writer']
)
writer_card = AgentCard(
    name="writer agent",
    description="Writer agent",
    skills=[writer_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
loop_skill = AgentSkill(
    id='loop-skill',
    name="loop skill",
    description="loop skill",
    tags=['loop']
)
loop_card = AgentCard(
    name="loop agent",
    description="loop agent",
    skills=[loop_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent = Agent(chain=chain, card=writer_card, state_change_callback=state_callback)
loop_agent = LoopAgent(agent=agent, is_stop=iter(max_iteration), card=loop_card, state_change_callback=state_callback)

In [4]:
query = "Write a short poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=-1)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running


agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a short poem with Shakespere's style about the peace before the storm." query_media=None query_media_type=None origin='loop agent' responses=[('writer agent', "O, fairest calm that doth descend upon\nThe troubled earth, like silence of a night,\nWhen s

In [5]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=1)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a poem with Shakespere's style about the peace before the storm." query_media=None query_media_

In [6]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=3)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a poem with Shakespere's style about the peace before the storm." query_media=None query_media_

In [7]:
def custom_keep_logic(responses: Sequence[AgentResponse]) -> List[AgentResponse]:
    return list(responses)


query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=custom_keep_logic)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running


agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a poem with Shakespere's style about the peace before the storm." query_media=None query_media_type=None origin='loop agent' responses=[('writer agent', 'O, fairest eve, when darkness doth descend\nAnd mortal souls do seek repose to lend,\nA fleeting c